<a href="https://colab.research.google.com/github/jungo876/Prac_ml/blob/main/BabyCryClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tensorflow_io

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 80.3 MB/s eta 0:00:00


In [4]:
import tensorflow as tf

In [5]:
import os
dataset_path = "/content/drive/MyDrive/babycry_dataset"
os.listdir(dataset_path)

['tried', 'hungry', 'discomfort', 'laugh']

In [6]:
import os
import tensorflow as tf

dataset_path = "/content/drive/MyDrive/babycry_dataset"

class_names = sorted(os.listdir(dataset_path))
print("Classes:", class_names)

Classes: ['discomfort', 'hungry', 'laugh', 'tried']


In [7]:
file_paths = []
labels = []

for label_index, class_name in enumerate(class_names):
    class_dir = os.path.join(dataset_path, class_name)
    for file_name in os.listdir(class_dir):
        if file_name.endswith("_fixed.wav"):
            file_paths.append(os.path.join(class_dir, file_name))
            labels.append(label_index)

In [8]:
file_paths = tf.constant(file_paths)
labels = tf.constant(labels)

ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))


In [9]:
def load_audio(path, label):
    audio_binary = tf.io.read_file(path)

    # TRY decoding and skip if invalid
    try:
        audio, sr = tf.audio.decode_wav(audio_binary)
    except:
        print("Skipping bad file:", path.numpy().decode())
        return tf.zeros([16000], dtype=tf.float32), -1  # dummy values

    audio = tf.squeeze(audio, axis=-1)
    desired_length = 16000

    audio = audio[:desired_length]
    padding = desired_length - tf.shape(audio)[0]
    audio = tf.pad(audio, [[0, padding]])

    return audio, label






In [10]:
MAX_FRAMES = 64

def extract_mfcc(audio, label):
    stft = tf.signal.stft(audio, frame_length=512, frame_step=256)
    spec = tf.abs(stft)

    mel_mat = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=40,
        num_spectrogram_bins=spec.shape[-1],
        sample_rate=16000
    )

    mel_spec = tf.tensordot(spec, mel_mat, 1)
    mel_spec = tf.math.log(mel_spec + 1e-6)

    mfcc = tf.signal.mfccs_from_log_mel_spectrograms(mel_spec)
    mfcc = mfcc[:, :40]

    mfcc = mfcc[:MAX_FRAMES]
    pad = MAX_FRAMES - tf.shape(mfcc)[0]
    mfcc = tf.pad(mfcc, [[0, pad], [0, 0]])

    return mfcc, label


mfcc_ds = ds.map(load_audio)
mfcc_ds = mfcc_ds.map(extract_mfcc)
mfcc_ds = mfcc_ds.map(lambda x, y: (tf.expand_dims(x, -1), y))
mfcc_ds = mfcc_ds.batch(32).prefetch(1)


In [12]:
for x, y in mfcc_ds.take(1):
    print(x)

tf.Tensor(
[[[[ 9.52334976e+00]
   [-4.33805609e+00]
   [ 2.49884024e-01]
   ...
   [ 2.71302108e-02]
   [ 4.46580172e-01]
   [-2.09500566e-01]]

  [[ 8.39665222e+00]
   [-4.93838453e+00]
   [-7.24532962e-01]
   ...
   [ 1.20420910e-01]
   [ 3.53642181e-02]
   [-4.04888421e-01]]

  [[ 8.21092701e+00]
   [-4.56981754e+00]
   [-2.47316733e-01]
   ...
   [ 2.03516155e-01]
   [ 3.65976930e-01]
   [-1.65065378e-01]]

  ...

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]

  [[ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   ...
   [ 0.00000000e+00]
   [ 0.00000000e+00]
   [ 0.00000000e+00]]]


 [[[-3.17052231e+01]
   [ 2.32764816e+00]
   [-5.80383897e-01]
   ...
   [-7.21090734e-02]
   [ 1.12244874e-01]
   [-8.35778862e-02]]

  [[-1.16990952e+02]
   [-9.2602437

In [10]:
# def add_channel(x, y):
#     x = tf.expand_dims(x, -1)
#     return x, y
#
# mfcc_ds = mfcc_ds.map(add_channel).batch(32)


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 40, 1)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [12]:
model.fit(mfcc_ds, epochs=20)


Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node sequential_1/flatten_1/Reshape defined at (most recent call last):
<stack traces unavailable>
only one input size may be -1, not both 0 and 1

Stack trace for op definition: 
File "<frozen runpy>", line 198, in _run_module_as_main
File "<frozen runpy>", line 88, in _run_code
File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
File "/usr/local/lib/python3.12/dist-packages/tornado/platform/asyncio.py", line 211, in start
File "/usr/lib/python3.12/asyncio/base_events.py", line 645, in run_forever
File "/usr/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once
File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
File "/usr/local/lib/python3.12/dist-packages/ipykernel/ker

In [ ]:
for mfcc, label in audio_ds.map(extract_mfcc).take(10):
    print(mfcc.shape)


(64, 40)
(64, 40)
(64, 40)
(64, 40)


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:3 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::ParallelMapV2::Filter::ParallelMapV2: Bad audio format for WAV: Expected 1 (PCM), but got3
	 [[{{node DecodeWav}}]] [Op:IteratorGetNext] name: 